In [20]:
import yaml
import pandas as pd
from functools import reduce
import numpy as np

def get_config():
    with open("config.yaml", 'r') as stream:
        config = yaml.safe_load(stream)
    return config

config = get_config()

def read_csv(yaml_key):
    filepath = (config[yaml_key])
    if not filepath.endswith('.csv'):
        raise Exception
    df = pd.read_csv(filepath)
    return df

conditions_df = read_csv('datapath_as3_conditions')
patients_df = read_csv('datapath_as3_patients')
observations_df = read_csv('datapath_as3_observations')
careplans_df = read_csv('datapath_as3_careplans')
encounters_df = read_csv('datapath_as3_encounters')
patients_df.head()


,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE
0,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,2017-08-24,NaN,999-68-6630,NaN,NaN,NaN,Jacinto644,Kris249,NaN,...,Beverly Massachusetts US,888 Hickle Ferry Suite 38,Springfield,Massachusetts,Hampden County,1106.0,42.151961,-72.598959,8446.49,1499.08
1,067318a4-db8f-447f-8b6e-f2f61e9baaa5,2016-08-01,NaN,999-15-5895,NaN,NaN,NaN,Alva958,Krajcik437,NaN,...,Boston Massachusetts US,1048 Skiles Trailer,Walpole,Massachusetts,Norfolk County,2081.0,42.177370,-71.281353,89893.40,1845.72
2,ae9efba3-ddc4-43f9-a781-f72019388548,1992-06-30,NaN,999-27-3385,S99971451,X53218815X,Mr.,Jayson808,Fadel536,NaN,...,Springfield Massachusetts US,1056 Harris Lane Suite 70,Chicopee,Massachusetts,Hampden County,1020.0,42.181642,-72.608842,577445.86,3528.84
3,199c586f-af16-4091-9998-ee4cfc02ee7a,2004-01-09,NaN,999-73-2461,S99956432,NaN,NaN,Jimmie93,Harris789,NaN,...,Worcester Massachusetts US,201 Mitchell Lodge Unit 67,Pembroke,Massachusetts,Plymouth County,NaN,42.075292,-70.757035,336701.72,2705.64
4,353016ea-a0ff-4154-85bb-1cf8b6cedf20,1996-11-15,NaN,999-60-7372,S99917327,X58903159X,Mr.,Gregorio366,Auer97,NaN,...,Patras Achaea GR,1050 Lindgren Extension Apt 38,Boston,Massachusetts,Suffolk County,2135.0,42.352434,-71.028610,484076.34,3043.04


In [21]:
# all ids are unique
patients_in_total = len(patients_df["Id"].unique())
print("There are", patients_in_total, 'unique patients in patients df out of', len(patients_df["Id"]))


unique_patients_in_conditions_df = conditions_df["PATIENT"].unique()
print("There are", len(unique_patients_in_conditions_df), 'unique patients in conditions df out of', len(conditions_df["PATIENT"]))
covid_patients_len = len(conditions_df[conditions_df['CODE']==840539006]["PATIENT"].unique())
print("There are", covid_patients_len, 'covid patients in conditions df out of', len(unique_patients_in_conditions_df))

# couldn't find value 'Hospital admission for isolation'
encounters_df.columns[encounters_df.isin(['Hospital admission for isolation']).any()]

# let's find correct description for 'Hospital admission for isolation' case
isolation_description = ''
possible_descriptions = encounters_df['DESCRIPTION'].unique()
for description in possible_descriptions:
    if 'isolation' in description:
        isolation_description = description
        print("Possible description:", description)
        break
        
hospital_isolation_patients = len(encounters_df[encounters_df['DESCRIPTION']==isolation_description]["PATIENT"].unique())
print("There are", hospital_isolation_patients, "patients who encountered hospital admission for isolation out of", len(encounters_df["PATIENT"].unique()))

# dead_patients = patients_df[patients_df['DEATHDATE'] != np.nan ]
dead_patients = patients_df['DEATHDATE'].where(~patients_df['DEATHDATE'].isna()).isnull().sum()
print("There are", dead_patients, 'dead patients out of', patients_in_total)


There are 12352 unique patients in patients df out of 12352
There are 12165 unique patients in conditions df out of 114544
There are 8820 covid patients in conditions df out of 12165
Possible description: Hospital admission for isolation (procedure)
There are 1867 patients who encountered hospital admission for isolation out of 12330
There are 10000 dead patients out of 12352


In [22]:
def part1(num_pat, num_cov, num_admitted, num_died):
    print(f'There are {num_pat} patients in total')
    print(f'There are {num_cov} covid patients')
    print(f'There are {num_admitted} admitted patients')
    print(f'{num_died} patients died')
    
part1(patients_in_total, covid_patients_len, hospital_isolation_patients, dead_patients)

There are 12352 patients in total
There are 8820 covid patients
There are 1867 admitted patients
10000 patients died


In [23]:
covid_patients = conditions_df[conditions_df['CODE']==840539006]
covid_patient_table = covid_patients.merge(patients_df, how='inner', left_on='PATIENT', right_on='Id')

codes = {'48065-7', '26881-3', '2276-4', '89579-7', '731-0', '14804-9'}

def code_mask(df):
    res = False
    for code in codes:
        res |= df['CODE'] == code
    return res
    
lab_obs_table = observations_df[(code_mask(observations_df))] 
covid_patient_table.head()

covid_patients_obs_table = covid_patient_table.merge(lab_obs_table, how='inner', on='PATIENT')
covid_patients_obs_table.head()

covid_patients_obs_table['DATE'] = pd.to_datetime(covid_patients_obs_table['DATE'], errors='coerce')
covid_patients_obs_table['START'] = pd.to_datetime(covid_patients_obs_table['START'], errors='coerce')
covid_patients_obs_table['days'] = covid_patients_obs_table['DATE'] - covid_patients_obs_table['START']

covid_patients_obs_table_cleared = covid_patients_obs_table[['PATIENT','days','CODE_y', 'VALUE']].copy()
covid_patients_obs_table_cleared = covid_patients_obs_table_cleared.rename( columns={'CODE_y':'CODE'})
covid_patients_obs_table_cleared["VALUE"] = covid_patients_obs_table_cleared["VALUE"].astype(float)

covid_patients_obs_table_cleared.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 73918 entries, 0 to 73917
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype          
---  ------   --------------  -----          
 0   PATIENT  73918 non-null  object         
 1   days     73918 non-null  timedelta64[ns]
 2   CODE     73918 non-null  object         
 3   VALUE    73918 non-null  float64        
dtypes: float64(1), object(2), timedelta64[ns](1)
memory usage: 2.8+ MB


In [24]:
#the following code is given, RUN THIS CELL
#get survived and deceased ids
completed_isolation_patients = careplans_df[(careplans_df.CODE == 736376001) & (careplans_df.STOP.notna()) \
                                          & (careplans_df.REASONCODE == 840539006)].PATIENT
negative_covid_patient_ids = observations_df[(observations_df.CODE == '94531-1') \
                                          & (observations_df.VALUE == 'Not detected (qualifier value)')].PATIENT.unique()
survivor_ids = np.union1d(completed_isolation_patients, negative_covid_patient_ids)
deceased_ids = patients_df[patients_df.DEATHDATE.notna()].Id

covid_patients_obs_table_cleared['IS_DEAD'] = covid_patients_obs_table_cleared['PATIENT'].isin(deceased_ids)
covid_patients_obs_table_cleared

,PATIENT,days,CODE,VALUE,IS_DEAD
0,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,0 days,731-0,1.1,False
1,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,0 days,48065-7,0.4,False
2,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,0 days,2276-4,332.4,False
3,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,0 days,89579-7,2.3,False
4,f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,0 days,14804-9,223.9,False
...,...,...,...,...,...
73913,c9699449-7a8b-400a-8e86-fab6aa7134cb,8 days,731-0,0.9,False
73914,c9699449-7a8b-400a-8e86-fab6aa7134cb,8 days,48065-7,0.5,False
73915,c9699449-7a8b-400a-8e86-fab6aa7134cb,8 days,2276-4,525.2,False
73916,c9699449-7a8b-400a-8e86-fab6aa7134cb,8 days,89579-7,3.0,False


In [25]:
def test3(survived, died):
    print(f'patients records survived: {survived}, patients records deceased {died}')
   
print('Total number of patients:', len(covid_patients_obs_table_cleared["PATIENT"].unique()))
test3(len(covid_patients_obs_table_cleared[covid_patients_obs_table_cleared['IS_DEAD'] == False].drop_duplicates('PATIENT')), len(covid_patients_obs_table_cleared[covid_patients_obs_table_cleared['IS_DEAD'] == True].drop_duplicates('PATIENT')))

Total number of patients: 1867
patients records survived: 1518, patients records deceased 349


In [26]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.io import output_notebook
from bokeh.plotting import ColumnDataSource
output_notebook()

Loading BokehJS ...

In [8]:
from bokeh.io import show
from bokeh.models import Dropdown
from bokeh.plotting import curdoc
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CategoricalColorMapper

def getPatientsWithCertainCode(df, code):
    return df[df['CODE'] == code]

def categorise(row):  
    if row['IS_DEAD']:
        return "deceased"
    return "survived"

covid_patients_obs_table_cleared['IS_DEAD_string'] = covid_patients_obs_table_cleared.apply(lambda row: categorise(row), axis=1)

defaultCode = '48065-7'
patientsWithCertainCode = getPatientsWithCertainCode(covid_patients_obs_table_cleared, defaultCode)

source = ColumnDataSource(
        data = {'days': patientsWithCertainCode['days'], 'VALUE': patientsWithCertainCode['VALUE'],
               'IS_DEAD_string': patientsWithCertainCode['IS_DEAD_string']}
    )

mapper = CategoricalColorMapper( factors=['deceased', 'survived'], palette=['red', 'green'])
pl = figure(plot_width=1600, plot_height=800, title='Interleukin 6 in Serum or Plasma', x_axis_type="datetime", x_axis_label='days', y_axis_label='pg/mL')
circles = pl.circle('days', 'VALUE', size=5, source=source, color={'field': 'IS_DEAD_string', 'transform': mapper}, legend_field='IS_DEAD_string')
ds = circles.data_source

def getMenu():
    return [("Code " + code, code) for code in codes]

def my_button_handler(new):
    patients = getPatientsWithCertainCode(covid_patients_obs_table_cleared, new.item)
    ds.data = data = {'days': patients['days'], 'VALUE': patients['VALUE'],
               'IS_DEAD_string': patients['IS_DEAD_string']}
    
dropdown = Dropdown(label="Select lab code", button_type="warning", menu=getMenu())
dropdown.on_click(my_button_handler)

curdoc().add_root(column(dropdown, pl))


In [58]:
import folium
m = folium.Map(location=[42.235325, -72.645978], zoom_start=7)

def addPatientMarker(lat, lon, name):
    global min_lat
    global min_lon
    marker = folium.Marker(
        location=[lat, lon],
        popup="<stong>"+name+"</stong>"
        )
    marker.add_to(m)  

lats = list(patients_df['LAT'])[:200]
lons = list(patients_df['LON'])[:200]
names = list(patients_df['FIRST'])[:200]
for i in range(0,len(lats)):
    addPatientMarker(lats[i], lons[i], names[i])
m

